# Task3_Ossan_Noémie_Yosr

## Lecture du fichier csv

In [130]:
import pandas as pd
import numpy as np
import csv

df = pd.read_csv('task3-train.csv', sep='\t', header=None, skiprows=10,quoting=csv.QUOTE_NONE)
print(len(df))
df1 = df.iloc[:,-2:] # get two lasts columns
df1.columns = ["text","sentiment"] # name columns


5118


## Séparation du corpus (80/20)

In [131]:
#Séparation
from sklearn.model_selection import train_test_split

data_train, data_test, sentiment_train, sentiment_test = train_test_split(df1.text, df1.sentiment, test_size=0.2,random_state=109) 


## Vectorization

In [132]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(data_train)
test_vectors = vectorizer.transform(data_test) # check difference
##il faut obtenir l'id du test, et la prediction

## Création du modèle à partir des données d'apprentissages

In [133]:
## Fonction analyse de résultats
def analyse(report):
    for key in report:
        print(key,'->',report[key])
        if key == 'accuracy':
            break

## Boucle1 SVM TFIDF sans nettoyage 

In [134]:
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')

#Train Data vectors
classifier_linear.fit(train_vectors, sentiment_train)


#Prediction
prediction_linear = classifier_linear.predict(test_vectors)

report = classification_report(sentiment_test, prediction_linear, output_dict=True)
analyse(report)

fullReport = {'Raw bag of word SVM' : report}

mixed -> {'precision': 0.22807017543859648, 'recall': 0.2, 'f1-score': 0.21311475409836067, 'support': 130}
negative -> {'precision': 0.6462882096069869, 'recall': 0.6548672566371682, 'f1-score': 0.6505494505494507, 'support': 452}
objective -> {'precision': 0.6781002638522428, 'recall': 0.7259887005649718, 'f1-score': 0.7012278308321964, 'support': 354}
positive -> {'precision': 0.5616438356164384, 'recall': 0.4659090909090909, 'f1-score': 0.5093167701863354, 'support': 88}
accuracy -> 0.60546875


## Création du modèle à partir du corpus complet

In [135]:
#Train on whole dataset
train_vectors = vectorizer.fit_transform(df1.text)
test_vector = vectorizer.transform(df1.text)
classifier_linear.fit(train_vectors, df1.sentiment)


#Prediction
prediction_linear = classifier_linear.predict(test_vector)

report = classification_report(df1.sentiment, prediction_linear, output_dict=True)

analyse(report)

mixed -> {'precision': 0.9984177215189873, 'recall': 0.9968404423380727, 'f1-score': 0.9976284584980237, 'support': 633}
negative -> {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2263}
objective -> {'precision': 0.9988365328679465, 'recall': 0.9994179278230501, 'f1-score': 0.999127145766657, 'support': 1718}
positive -> {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 504}
accuracy -> 0.9994138335287222


## Run2 with TFIDF / SVM


In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer1 = TfidfVectorizer(min_df = 1,max_df = 0.9,sublinear_tf = True,use_idf = True)
train_vectors = vectorizer1.fit_transform(data_train)
test_vectors = vectorizer1.transform(data_test) # check difference
##il faut obtenir l'id du test, et la prediction
# Perform classification with SVM, kernel=linear
classifier_linear1 = svm.SVC(kernel='linear')

#Train Data vectors
classifier_linear1.fit(train_vectors, sentiment_train)


#Prediction
prediction_linear = classifier_linear1.predict(test_vectors)

report = classification_report(sentiment_test, prediction_linear, output_dict=True)
analyse(report)
fullReport['Raw tfidf SVM'] = report

mixed -> {'precision': 0.3125, 'recall': 0.038461538461538464, 'f1-score': 0.06849315068493152, 'support': 130}
negative -> {'precision': 0.6378091872791519, 'recall': 0.7986725663716814, 'f1-score': 0.7092337917485265, 'support': 452}
objective -> {'precision': 0.7097625329815304, 'recall': 0.7598870056497176, 'f1-score': 0.7339699863574352, 'support': 354}
positive -> {'precision': 0.6190476190476191, 'recall': 0.4431818181818182, 'f1-score': 0.5165562913907285, 'support': 88}
accuracy -> 0.658203125


## Full Raw

In [137]:
#Train on whole dataset
train_vectorsFinal = vectorizer1.fit_transform(df1.text)
test_vectorsFinal = vectorizer1.transform(df1.text)

classifier_linear1.fit(train_vectorsFinal, df1.sentiment)
#Prediction
prediction_linear = classifier_linear1.predict(test_vectorsFinal)
# Résultat
report = classification_report(df1.sentiment, prediction_linear, output_dict=True)
analyse(report)

mixed -> {'precision': 0.9705882352941176, 'recall': 0.46919431279620855, 'f1-score': 0.6325878594249201, 'support': 633}
negative -> {'precision': 0.8234415826801045, 'recall': 0.9748121961997349, 'f1-score': 0.8927559692432213, 'support': 2263}
objective -> {'precision': 0.9225393127548049, 'recall': 0.9220023282887078, 'f1-score': 0.9222707423580786, 'support': 1718}
positive -> {'precision': 0.9567307692307693, 'recall': 0.7896825396825397, 'f1-score': 0.8652173913043478, 'support': 504}
accuracy -> 0.8763188745603752


## Run3 Stop words (French dictionary) / with TFIDF / SVM

In [138]:
import nltk
import string
from nltk.corpus import stopwords
stop_words_list = stopwords.words('french')
# max_df ignorer les mot qui apparaisent dans plus de 90% du document

vectorizer2 = TfidfVectorizer(min_df = 1,max_df = 0.9,sublinear_tf = True,use_idf = True, stop_words=stop_words_list)

train_vectors = vectorizer2.fit_transform(data_train)
test_vectors = vectorizer2.transform(data_test) # check difference
##il faut obtenir l'id du test, et la prediction
# Perform classification with SVM, kernel=linear
classifier_linear2 = svm.SVC(kernel='linear')

#Train Data vectors
classifier_linear2.fit(train_vectors, sentiment_train)


#Prediction
prediction_linear = classifier_linear2.predict(test_vectors)

report = classification_report(sentiment_test, prediction_linear, output_dict=True)
analyse(report)
fullReport['StopWords tfidf SVM'] = report

data_train

mixed -> {'precision': 0.25, 'recall': 0.03076923076923077, 'f1-score': 0.05479452054794521, 'support': 130}
negative -> {'precision': 0.6309090909090909, 'recall': 0.7676991150442478, 'f1-score': 0.6926147704590818, 'support': 452}
objective -> {'precision': 0.6910994764397905, 'recall': 0.7457627118644068, 'f1-score': 0.7173913043478262, 'support': 354}
positive -> {'precision': 0.5789473684210527, 'recall': 0.5, 'f1-score': 0.5365853658536586, 'support': 88}
accuracy -> 0.6435546875


1941    La plus grande erreur que jai faite durant ma ...
1688    La Russie crée un ministère chargé de la Crimé...
1045    Lorsque que Obama aura une rencontre avec Maro...
3087    "Avons nous des nouvelles de Léonarda?" Dernie...
1198    Tjs qd qu'il est au cabinet du Min. de la Cult...
                              ...                        
2635    #Hollande annonce une réduction d'impôts ... s...
2447    Racisme, chantages,... Des raisons pour ne plu...
141     "Je n'accepterai pas de leçons de morale d'une...
3317    Actu. Afrique - Cameroun : les États-Unis dépl...
3334    Quand Barack Obama débarque dans une cérémonie...
Name: text, Length: 4094, dtype: object

In [139]:
#Train on whole dataset
train_vectorsFinal = vectorizer2.fit_transform(df1.text)
test_vectorsFinal = vectorizer2.transform(df1.text)

classifier_linear2.fit(train_vectorsFinal, df1.sentiment)
#Prediction
prediction_linear = classifier_linear2.predict(test_vectorsFinal)
# Résultat
report = classification_report(df1.sentiment, prediction_linear, output_dict=True)
analyse(report)

mixed -> {'precision': 0.9850299401197605, 'recall': 0.5197472353870458, 'f1-score': 0.6804550155118926, 'support': 633}
negative -> {'precision': 0.8333333333333334, 'recall': 0.9721608484312859, 'f1-score': 0.8974097491331837, 'support': 2263}
objective -> {'precision': 0.9253903990746096, 'recall': 0.9313154831199069, 'f1-score': 0.9283434870902234, 'support': 1718}
positive -> {'precision': 0.9566265060240964, 'recall': 0.7876984126984127, 'f1-score': 0.8639825897714909, 'support': 504}
accuracy -> 0.8843298163345057


## Prétraitements

In [140]:
#Prétraitements de données, suppression de stopwords avec lemmatization and stemming
from nltk.stem.snowball import FrenchStemmer
import spacy
import re

nlp = spacy.load("fr_core_news_sm")
stemmer = FrenchStemmer()

def pretraitement(text):
    # Lowercase
    text = text.lower()
    # Remove url
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    # Remove punctuation
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    text = tokenizer.tokenize(text)
    # Remove French stopwords and punctuation 
    tokenized_word = text
    filtered_sent=[]
    for w in tokenized_word:
        if w not in stop_words_list:
            filtered_sent.append(w)
    text = ' '.join(filtered_sent)  
    # lemmatization and stemming
    return " ".join([stemmer.stem(token.lemma_) for token in nlp(text)])

# Remplacement par le dataFrame propre
df1['text'] = df1['text'].to_frame().applymap(pretraitement)


# Séparation du corpus prétraité
data_train, data_test, sentiment_train, sentiment_test = train_test_split(df1.text, df1.sentiment, test_size=0.2,random_state=109)




## SVM TFIDF NGramms

In [141]:
# Principale Vectorizer
vectorizer = TfidfVectorizer(min_df = 1,max_df = 0.9,sublinear_tf = True,use_idf = True, ngram_range=(1,3))

train_vectors = vectorizer.fit_transform(data_train)
test_vectors = vectorizer.transform(data_test) 

##il faut obtenir l'id du test, et la prediction
# Perform classification with SVM, kernel=linear
svm_linear = svm.SVC(kernel='linear')
#Train Data vectors
svm_linear.fit(train_vectors, sentiment_train)
#Prediction
prediction_linear = svm_linear.predict(test_vectors)
# Résultat
report = classification_report(sentiment_test, prediction_linear, output_dict=True)
analyse(report)
fullReport['Clean tfidf SVM Ngram'] = report

data_train

mixed -> {'precision': 0.5, 'recall': 0.023076923076923078, 'f1-score': 0.04411764705882353, 'support': 130}
negative -> {'precision': 0.6070234113712375, 'recall': 0.8030973451327433, 'f1-score': 0.6914285714285714, 'support': 452}
objective -> {'precision': 0.6835820895522388, 'recall': 0.6468926553672316, 'f1-score': 0.6647314949201742, 'support': 354}
positive -> {'precision': 0.5294117647058824, 'recall': 0.5113636363636364, 'f1-score': 0.5202312138728323, 'support': 88}
accuracy -> 0.625


1941    plus grand erreur jai fair dur scolarit demand...
1688            russ cré minister charg crim soviet oeuvr
1045          lorsqu obam rencontr marois poul dent polqc
3087    nouveau léonard derni mot françois holland ava...
1198    tj qd cabinet min cultur sous sarkozy mathieu ...
                              ...                        
2635    holland annonc réduct impôt sûr 0 1 dir cel av...
2447    racism chantag raison plus lir jeun afriqu mic...
141     accept leçon moral gauch vouloir install dsk e...
3317    actu afriqu cameroun état unir déploient 300 m...
3334    quand barack obam débarqu cérémon mariag vi la...
Name: text, Length: 4094, dtype: object

## Full Run

In [142]:
#Train on whole dataset
train_vectorsFinal = vectorizer.fit_transform(df1.text)
test_vectorsFinal = vectorizer.transform(df1.text)

svm_linear.fit(train_vectorsFinal, df1.sentiment)
#Prediction
prediction_linear = svm_linear.predict(test_vectorsFinal)
# Résultat
report = classification_report(df1.sentiment, prediction_linear, output_dict=True)
analyse(report)

mixed -> {'precision': 0.9983471074380166, 'recall': 0.9541864139020537, 'f1-score': 0.975767366720517, 'support': 633}
negative -> {'precision': 0.9821038847664775, 'recall': 0.9942554131683606, 'f1-score': 0.9881422924901185, 'support': 2263}
objective -> {'precision': 0.9816091954022989, 'recall': 0.9941792782305006, 'f1-score': 0.9878542510121459, 'support': 1718}
positive -> {'precision': 0.995850622406639, 'recall': 0.9523809523809523, 'f1-score': 0.973630831643002, 'support': 504}
accuracy -> 0.9851504493942946


## baiyes TFIDF NGramms

In [143]:
from sklearn.naive_bayes import GaussianNB
#création d'un classifieur Gaussian
gnb = GaussianNB()
#entrainement
gnb.fit(train_vectors.toarray(), sentiment_train)
#Prediction
prediction_linear = gnb.predict(test_vectors.toarray())
# Résultat
report = classification_report(sentiment_test, prediction_linear, output_dict=True)
analyse(report)
fullReport['Clean tfidf baiyes Ngram'] = report

mixed -> {'precision': 0.13861386138613863, 'recall': 0.1076923076923077, 'f1-score': 0.12121212121212122, 'support': 130}
negative -> {'precision': 0.6033057851239669, 'recall': 0.6460176991150443, 'f1-score': 0.623931623931624, 'support': 452}
objective -> {'precision': 0.6303724928366762, 'recall': 0.6214689265536724, 'f1-score': 0.6258890469416786, 'support': 354}
positive -> {'precision': 0.4, 'recall': 0.4090909090909091, 'f1-score': 0.40449438202247195, 'support': 88}
accuracy -> 0.548828125


## Full Naive

In [144]:
#entrainement
gnb.fit(train_vectorsFinal.toarray(), df1.sentiment)
#Prediction
prediction_linear = gnb.predict(test_vectorsFinal.toarray())
# Résultat
report = classification_report(df1.sentiment, prediction_linear, output_dict=True)
analyse(report)

mixed -> {'precision': 0.9890453834115805, 'recall': 0.9984202211690363, 'f1-score': 0.9937106918238993, 'support': 633}
negative -> {'precision': 1.0, 'recall': 0.999116217410517, 'f1-score': 0.9995579133510168, 'support': 2263}
objective -> {'precision': 0.9994158878504673, 'recall': 0.9959254947613504, 'f1-score': 0.9976676384839651, 'support': 1718}
positive -> {'precision': 0.9960474308300395, 'recall': 1.0, 'f1-score': 0.998019801980198, 'support': 504}
accuracy -> 0.9980461117624072


## KNN TFIDF unigrams bigramm, trigrams

In [145]:
from sklearn.neighbors import KNeighborsClassifier
# création d'un classifieur Gaussian
knn = KNeighborsClassifier(n_neighbors=5)
# entrainement
knn.fit(train_vectors, sentiment_train)
# Prediction
prediction_linear = knn.predict(test_vectors.toarray())
# Résultat
report = classification_report(sentiment_test, prediction_linear, output_dict=True)
analyse(report)
fullReport['Clean tfidf KNN Ngram'] = report

mixed -> {'precision': 0.17105263157894737, 'recall': 0.1, 'f1-score': 0.1262135922330097, 'support': 130}
negative -> {'precision': 0.6206896551724138, 'recall': 0.5973451327433629, 'f1-score': 0.608793686583991, 'support': 452}
objective -> {'precision': 0.5815602836879432, 'recall': 0.6949152542372882, 'f1-score': 0.6332046332046332, 'support': 354}
positive -> {'precision': 0.45555555555555555, 'recall': 0.4659090909090909, 'f1-score': 0.4606741573033708, 'support': 88}
accuracy -> 0.556640625


## full knn

In [146]:
#entrainement
knn.fit(train_vectorsFinal, df1.sentiment)
#Prediction
prediction_linear = knn.predict(test_vectorsFinal)
# Résultat
report = classification_report(df1.sentiment, prediction_linear, output_dict=True)
analyse(report)

mixed -> {'precision': 0.558252427184466, 'recall': 0.36334913112164297, 'f1-score': 0.4401913875598086, 'support': 633}
negative -> {'precision': 0.7573626373626373, 'recall': 0.7613787008395935, 'f1-score': 0.7593653591890701, 'support': 2263}
objective -> {'precision': 0.7026627218934911, 'recall': 0.829452852153667, 'f1-score': 0.7608115323011211, 'support': 1718}
positive -> {'precision': 0.7245657568238213, 'recall': 0.5793650793650794, 'f1-score': 0.6438809261300993, 'support': 504}
accuracy -> 0.7170769831965611


## Reports

In [147]:
fullReport = pd.DataFrame.from_dict(fullReport)
fullReport

,Raw bag of word SVM,Raw tfidf SVM,StopWords tfidf SVM,Clean tfidf SVM Ngram,Clean tfidf baiyes Ngram,Clean tfidf KNN Ngram
mixed,"{'precision': 0.22807017543859648, 'recall': 0...","{'precision': 0.3125, 'recall': 0.038461538461...","{'precision': 0.25, 'recall': 0.03076923076923...","{'precision': 0.5, 'recall': 0.023076923076923...","{'precision': 0.13861386138613863, 'recall': 0...","{'precision': 0.17105263157894737, 'recall': 0..."
negative,"{'precision': 0.6462882096069869, 'recall': 0....","{'precision': 0.6378091872791519, 'recall': 0....","{'precision': 0.6309090909090909, 'recall': 0....","{'precision': 0.6070234113712375, 'recall': 0....","{'precision': 0.6033057851239669, 'recall': 0....","{'precision': 0.6206896551724138, 'recall': 0...."
objective,"{'precision': 0.6781002638522428, 'recall': 0....","{'precision': 0.7097625329815304, 'recall': 0....","{'precision': 0.6910994764397905, 'recall': 0....","{'precision': 0.6835820895522388, 'recall': 0....","{'precision': 0.6303724928366762, 'recall': 0....","{'precision': 0.5815602836879432, 'recall': 0...."
positive,"{'precision': 0.5616438356164384, 'recall': 0....","{'precision': 0.6190476190476191, 'recall': 0....","{'precision': 0.5789473684210527, 'recall': 0....","{'precision': 0.5294117647058824, 'recall': 0....","{'precision': 0.4, 'recall': 0.409090909090909...","{'precision': 0.45555555555555555, 'recall': 0..."
accuracy,0.605469,0.658203,0.643555,0.625,0.548828,0.556641
macro avg,"{'precision': 0.5285256211285662, 'recall': 0....","{'precision': 0.5697798348270753, 'recall': 0....","{'precision': 0.5377389839424835, 'recall': 0....","{'precision': 0.5800043164073396, 'recall': 0....","{'precision': 0.44307303483669547, 'recall': 0...","{'precision': 0.457214531498715, 'recall': 0.4..."
weighted avg,"{'precision': 0.5969175239133946, 'recall': 0....","{'precision': 0.6197723435564736, 'recall': 0....","{'precision': 0.5988940353238745, 'recall': 0....","{'precision': 0.6132332782572358, 'recall': 0....","{'precision': 0.5361971477738423, 'recall': 0....","{'precision': 0.5358884722242334, 'recall': 0...."


### Tests avec Tweet non annotés

In [148]:
##Creation a predictioncsv file
def predict(inputFile,outputFile, vectorizer, model):
    ## Get DataFrame from csv file (column => id tweet)
    dfTest = pd.read_csv('task3-test.csv',
                   delimiter='\t',
                   header=None,
                   names=['tweet_x', 'sentiment_x'],quoting=csv.QUOTE_NONE)

    dfTestTweet = dfTest.iloc[:,-1:] # get the last column => tweet
    dfTestTweet = dfTestTweet.iloc[:,0] # Turn into panda Series
    
    #Vecteur des ids numpy array str
    dfTestId = ((dfTest.iloc[:,0]).to_numpy()).astype(str)
    #Création du vecteur de données tests
    dfTestTweet = dfTestTweet.to_frame().applymap(pretraitement)
    # convert to series for the vectorizer
    dfTestTweet.columns = ['data']
    test_vectors = vectorizer.transform(dfTestTweet['data']) 
    
    #Prediction
    if model == gnb:
       prediction = model.predict(test_vectors.toarray()) 
    else:
        prediction = model.predict(test_vectors)
   
    #Create file lines 
    for i in range(dfTestId.size):
        dfTestId[i]= "(" + dfTestId[i] + ")"
        
    matrix = np.column_stack((prediction,dfTestId))
    
    #Create the file
    np.savetxt(outputFile, matrix ,fmt='%s')


In [151]:
# MDP => Changer le nom du fichier csv et le classifier
predict('task3-test.csv','task3_KassegneOssan_run1.sc', vectorizer,svm_linear)
predict('task3-test.csv','task3_KassegneOssan_run2.sc', vectorizer1,classifier_linear1)
predict('task3-test.csv','task3_KassegneOssan_run3.sc', vectorizer2,classifier_linear2)